# Apply Stardist to large image in OMERO

In [1]:
from skimage.io import imread
import numpy as np
import matplotlib.pyplot as plt
import napari
from napari.utils import nbscreenshot

from stardist.models import StarDist2D
from csbdeep.utils import normalize
from stardist.big import BlockND
from stardist.matching import relabel_sequential

from omero.gateway import BlitzGateway
from getpass import getpass
import ezomero
import matplotlib.pyplot as plt

numpy == 1.23.5

## Connect to OMERO server

In [2]:
HOST = 'omero-int.biotec.tu-dresden.de'

conn = ezomero.connect(host=HOST, user=input("Username: "),
               password=getpass("OMERO Password: "), port=4064)

## Get image object

Choose image ID

In [4]:
image_ids = [416]

Get image object and parent dataset

In [5]:
# get image object
image_id = image_ids[0]
# Read image object (without getting the pixels)
omero_image, image = ezomero.get_image(conn, image_id, no_pixels=True) # shape follows TZYXC
# get dataset where image comes from
dataset = omero_image.getParent()
print("From Dataset: ", dataset.getName())
# Get dataset ID
dataset_id = dataset.getId()

From Dataset:  from_Ellen


Getting image data in one shot is not possible with large images due to memory error (expand below to know more)

In [7]:
# omero_image, image = ezomero.get_image(conn, image_id, no_pixels=False)

<details>
  <summary>Error message</summary>
    
   ```python
    ---------------------------------------------------------------------------
UnknownLocalException                     Traceback (most recent call last)
c:\Users\mazo260d\Documents\GitHub\omero-python\notebooks\apply_stardist_to_tiled_omero_image.ipynb Cell 12 line 1
----> 1 omero_image, image = ezomero.get_image(conn, image_id, no_pixels=False)

File c:\Users\mazo260d\mambaforge\envs\omero-python-env\lib\site-packages\ezomero\_ezomero.py:63, in do_across_groups.<locals>.wrapper(*args, **kwargs)
     61     current_group = args[0].getGroupFromContext().getId()
     62     args[0].SERVICE_OPTS.setOmeroGroup('-1')
---> 63     res = f(*args, **kwargs)
     64     set_group(args[0], current_group)
     65 else:

File c:\Users\mazo260d\mambaforge\envs\omero-python-env\lib\site-packages\ezomero\_gets.py:207, in get_image(conn, image_id, no_pixels, start_coords, axis_lengths, xyzct, pad, pyramid_level, dim_order, across_groups)
    204         zct_tiles.append((zct[0], zct[1], zct[2], tile))
    205     plane_gen = primary_pixels.getTiles(zct_tiles)
--> 207 for i, plane in enumerate(plane_gen):
    208     zct_coords = zct_list[i]
    209     z = zct_coords[0] - start_coords[2]

File c:\Users\mazo260d\mambaforge\envs\omero-python-env\lib\site-packages\omero\gateway\__init__.py:7561, in _PixelsWrapper.getTiles(self, zctTileList)
   7559         exc = e
   7560 if exc is not None:
-> 7561     raise exc

File c:\Users\mazo260d\mambaforge\envs\omero-python-env\lib\site-packages\omero\gateway\__init__.py:7528, in _PixelsWrapper.getTiles(self, zctTileList)
   7526 z, c, t, tile = zctTile
   7527 if tile is None:
-> 7528     rawPlane = rawPixelsStore.getPlane(z, c, t)
   7529     planeY = sizeY
   7530     planeX = sizeX

File c:\Users\mazo260d\mambaforge\envs\omero-python-env\lib\site-packages\omero\gateway\__init__.py:4858, in OmeroGatewaySafeCallWrapper.__call__(self, *args, **kwargs)
   4856 except Exception as e:
   4857     self.debug(e.__class__.__name__, args, kwargs)
-> 4858     return self.handle_exception(e, *args, **kwargs)

File c:\Users\mazo260d\mambaforge\envs\omero-python-env\lib\site-packages\omero\gateway\__init__.py:4855, in OmeroGatewaySafeCallWrapper.__call__(self, *args, **kwargs)
   4853 def __call__(self, *args, **kwargs):
   4854     try:
-> 4855         return self.f(*args, **kwargs)
   4856     except Exception as e:
   4857         self.debug(e.__class__.__name__, args, kwargs)

File c:\Users\mazo260d\mambaforge\envs\omero-python-env\lib\site-packages\omero_api_RawPixelsStore_ice.py:1199, in RawPixelsStorePrx.getPlane(self, z, c, t, _ctx)
   1198 def getPlane(self, z, c, t, _ctx=None):
-> 1199     return _M_omero.api.RawPixelsStore._op_getPlane.invoke(self, ((z, c, t), _ctx))

UnknownLocalException: exception ::Ice::UnknownLocalException
{
    unknown = ConnectionI.cpp:1573: Ice::MemoryLimitException:
protocol error: memory limit exceeded:
requested 823353042 bytes, maximum allowed is 256000000 bytes (see Ice.MessageSizeMax)
  0 IceUtil::Exception::Exception(char const*, int) in /opt/ice-3.6.5/bin/../lib64/libIceUtil.so.36
  1 Ice::LocalException::LocalException(char const*, int) in /opt/ice-3.6.5/bin/../lib64/libIce.so.36
  2 Ice::ProtocolException::ProtocolException(char const*, int, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&) in /opt/ice-3.6.5/bin/../lib64/libIce.so.36
  3 Ice::MarshalException::MarshalException(char const*, int, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&) in /opt/ice-3.6.5/bin/../lib64/libIce.so.36
  4 Ice::MemoryLimitException::MemoryLimitException(char const*, int, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&) in /opt/ice-3.6.5/bin/../lib64/libIce.so.36
  5 IceInternal::Ex::throwMemoryLimitException(char const*, int, unsigned long, unsigned long) in /opt/ice-3.6.5/bin/../lib64/libIce.so.36
  6 /opt/ice-3.6.5/bin/../lib64/libIce.so.36(+0x1d7e3a) [0x7fbe2c3cce3a]
  7 /opt/ice-3.6.5/bin/../lib64/libIce.so.36(+0x3953e6) [0x7fbe2c58a3e6]
  8 /opt/ice-3.6.5/bin/../lib64/libIce.so.36(+0x397a98) [0x7fbe2c58ca98]
  9 /opt/ice-3.6.5/bin/../lib64/libIceUtil.so.36(+0x642f4) [0x7fbe2c1cd2f4]
 10 /lib/x86_64-linux-gnu/libpthread.so.0(+0x8609) [0x7fbe2beb8609]
 11 clone() in /lib/x86_64-linux-gnu/libc.so.6
}
   ```

</details>

Display image shape and levels (if applicable)

In [5]:
image_shape = (omero_image.getSizeT(), omero_image.getSizeZ(), omero_image.getSizeY(), omero_image.getSizeX(), omero_image.getSizeC())
image_shape

(1, 1, 18549, 22194, 1)

If image is in pyramidal structure, display pyramid levels

In [6]:
# Get pyramid levels
try:
    levels = ezomero.get_pyramid_levels(conn, image_id)
except AttributeError:
    levels = []
levels

[(22194, 18549),
 (11097, 9275),
 (5549, 4637),
 (2774, 2319),
 (1387, 1159),
 (694, 580)]

Stardist has a [`predict_instances_big`](https://github.com/stardist/stardist/blob/c0564459e344b62d5b50db83546de6b478db890c/stardist/models/base.py#L827) method, which tiles the images, run predictions and connects labelled tiles.

It is not possible to directly use it here because we don't have access to the full image from OMERO.
We need to get the tiles exactly how stardist expects them to be. So, we create the stardist Block object below.

Create Stardist Block object (Tiler)

In [7]:
# If image is in pyramid structure, apply stardist to tiles of highest resolution
if (levels) and (len(levels) > 1):
    # Set up some stardist block arguments
    shape = image_shape[-3:-1] # (y, x) from (t, z, y, x, c)
    block_size = min(levels[-3])
    min_overlap = int(block_size//10)
    context = min_overlap
    grid = 1
    print('block_size = ', block_size)
    axes = 'YX'
    # create block cover
    blocks = BlockND.cover(
        shape, axes, block_size, min_overlap, context=min_overlap, grid=1)
    

block_size =  2319


Set Stardist 2D Pre-trained Model

In [8]:
model = StarDist2D.from_pretrained('2D_versatile_fluo')

Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


Read blocks

In [9]:
image_id

416

**TO DO: check possible error with padding**

## Apply stardist to tiles
The code below:
- creates an empty output array
- reads tiles using ezomero
- the tiles are generated with stardist Block object
- pads the tile in case it is on the edges
- predict instances in tiles
- filter objects (match labels on tile edges)
- relabels them sequentially
- write tile to output

In [12]:
# Create empty output for labels
labels = np.zeros(shape, dtype=np.uint32)
label_offset = 1
# Set channel
ch = 0
for block in blocks:
    # Get tile from block_slices (in OMERO, these slices should be used to ezomero.get_image's start_coords and axis_lengths arguments )
    slc = block.slice_read()
    y_start, y_stop = slc[0].start, slc[0].stop
    x_start, x_stop = slc[1].start, slc[1].stop
    print('\nstart_coords = ', y_start, x_start)
    print('\nstop_coords = ', y_stop, x_stop)

    x_axis_length = min(x_stop - x_start, image_shape[-2] - x_start - 1)
    y_axis_length = min(y_stop - y_start, image_shape[-3] - y_start - 1)
    print('\naxis_lengths = ', y_axis_length, x_axis_length)
    _, tile = ezomero.get_image(conn,
                                image_id,
                                start_coords=(
                                    x_start, y_start, 0, ch, 0),
                                axis_lengths=(
                                    x_axis_length, y_axis_length, 1, 1, 1),
                                dim_order='yxzct',
                                pad = False,
                                )
    tile = tile.squeeze()
    print(tile.shape)
    tile2d = np.pad(tile, ((0, block_size - y_axis_length), (0, block_size - x_axis_length)), 'constant')
    print(tile2d.shape)
    # if tile.shape != tile2d.shape:
    #     print('Tile was reshaped')
    #     break
    # Perform local predictions to normalized tile
    labels_local, polys = model.predict_instances(normalize(tile2d))
    # Filter objects (match edges)
    labels_local = block.crop_context(labels_local)
    labels_local, polys = block.filter_objects(labels_local, polys)
    # Relabel labels sequentially from last offset
    labels_local = relabel_sequential(labels_local, label_offset)[0]
    # Write processed block (labels) to
    block.write(labels, labels_local)
    # Update labels offset
    label_offset += len(polys['prob'])


start_coords =  0 0

stop_coords =  2319 2319

axis_lengths =  2319 2319
(2319, 2319)
(2319, 2319)

start_coords =  0 1528

stop_coords =  2319 3847

axis_lengths =  2319 2319
(2319, 2319)
(2319, 2319)

start_coords =  0 3056

stop_coords =  2319 5375

axis_lengths =  2319 2319
(2319, 2319)
(2319, 2319)

start_coords =  0 4585

stop_coords =  2319 6904

axis_lengths =  2319 2319
(2319, 2319)
(2319, 2319)

start_coords =  0 6114

stop_coords =  2319 8433

axis_lengths =  2319 2319
(2319, 2319)
(2319, 2319)

start_coords =  0 7643

stop_coords =  2319 9962

axis_lengths =  2319 2319
(2319, 2319)
(2319, 2319)

start_coords =  0 9172

stop_coords =  2319 11491

axis_lengths =  2319 2319
(2319, 2319)
(2319, 2319)

start_coords =  0 10701

stop_coords =  2319 13020

axis_lengths =  2319 2319
(2319, 2319)
(2319, 2319)

start_coords =  0 12230

stop_coords =  2319 14549

axis_lengths =  2319 2319
(2319, 2319)
(2319, 2319)

start_coords =  0 13759

stop_coords =  2319 16078

axis_lengths =  23

## Temporarily saving labels locally

In [17]:
from skimage import io
# io.imsave('../data/Ellen_labels.tif', labels)

2149286119.py (2): ../data/Ellen_labels.tif is a low contrast image


# TODO: import labels back to OMERO same dataset

The suggetsed way so far to do this is to save the output as ome-tif or ome-zarr and upload it to omero using CLI or a script similar to [here](https://forum.image.sc/t/how-to-write-tiles-from-large-numpy-image-to-omero/79756/5) and [here](https://gitlab.com/openmicroscopy/incubator/omero-python-importer/-/blob/master/import.py).

## Close the connection to OMERO afterwards

In [13]:
conn.close()